In [77]:
! pip install python-dotenv

3344.82s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [78]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import gdown
import os
from dotenv import load_dotenv

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [79]:
ROOT_DIR: str = os.path.dirname(os.path.abspath(os.curdir))
DATASET_PATH: str = f"{ROOT_DIR}/source_data/superset_hr.xlsx"
GOOGLE_COLAB_PATH: str = '/content/superset_hr.xlsx'
load_dotenv()

if DATASET_URL := os.environ.get('DATASET_URL') is None:
    DATASET_URL = input('>>> .env was not found! Provide url for downloading: ')

def get_original_dataframe() -> pd.DataFrame:
    try:
        df = pd.read_excel(DATASET_PATH)
    except FileNotFoundError:
        try:
            print(f"Can't open file from path: {DATASET_PATH}", end='\n\n')
            df = pd.read_excel(GOOGLE_COLAB_PATH)
        except FileNotFoundError:
            print(f"Can't open file from path: {GOOGLE_COLAB_PATH}", end='\n\n')
            file_name = gdown.download(DATASET_URL, fuzzy=True)
            os.makedirs(os.path.dirname(DATASET_PATH), exist_ok=True)
            os.rename(os.path.abspath(file_name), DATASET_PATH)
            print(f'Moved to: {DATASET_PATH}', end='\n\n')
            df = pd.read_excel(DATASET_PATH)
    finally:
        print('Success!')
        return df

In [80]:
df_original = get_original_dataframe()

Success!


In [81]:
print(df_original.shape)
df_original.sample(3)

(175455, 24)


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,income_name,name,published_at,created_at,collected_at,url,...,address__city,schedule__name,grade,employment__name,key_skills__names,languages__names,exchange_rate,salary_from_gross,salary_to_gross,salary_average
96370,98085,98085,98084,95130439,DevOps,DevOps engineer,2024-03-20T14:10:51+0300,2024-03-20T14:10:51+0300,2024-03-24 19:26:33.262,https://api.hh.ru/vacancies/95130439?host=hh.ru,...,NaN,Удаленная работа,Junior (1-3),Полная занятость,"['Linux', 'Kubernetes', 'DevOps', 'Ansible', '...",[],NaN,NaN,NaN,NaN
106107,107932,107932,107931,95493605,Project manager,Руководитель проекта,2024-03-26T06:52:15+0300,2024-03-26T06:52:15+0300,2024-03-29 13:42:32.044,https://api.hh.ru/vacancies/95493605?host=hh.ru,...,Омск,Полный день,Middle (3-6),Полная занятость,"['системность', 'Исполнительность', 'Коммуника...",[],1.0,169500.0,226000.0,197750.0
42410,43016,43016,43016,94245916,Системный аналитик,Системный аналитик,2024-03-05T08:51:53+0300,2024-03-05T08:51:53+0300,2024-03-06 19:17:01.413,https://api.hh.ru/vacancies/94245916?host=hh.ru,...,NaN,Полный день,Junior (1-3),Полная занятость,"['Atlassian Confluence', 'Atlassian Jira', 'UM...",[],NaN,NaN,NaN,NaN


In [82]:
def print_df_info(df: pd.DataFrame) -> None:
    print(f"Shape: {df.shape}")
    print('-' * 50)
    display(df.sample(3))
    print('-' * 50)
    display(df.info())
    print('-' * 50)
    display(df.isna().sum())
    print('-' * 50)


def get_clear_df_version(df: pd.DataFrame) -> pd.DataFrame:
    df_new = df[['id', 'income_name', 'area__name', 'schedule__name', 'grade',
                 'key_skills__names', 'salary__currency', 'employer__name', 'languages__names',
                 'exchange_rate', 'salary_from_gross', 'salary_to_gross', 'salary_average']].copy()

    df_new = df_new.dropna(subset=['salary_from_gross', 'salary_to_gross', 'salary_average'], how='all')
    df_new = df_new.dropna(subset=['income_name'])

    df_new['not_rur'] = df_new['exchange_rate'] != 1
    df_new['not_rur'] = df_new['not_rur'].astype(int)
    df_new = df_new.drop(['salary__currency'], axis=1)

    return df_new


def get_difference_percentiles(df: pd.DataFrame) -> tuple[float, float, float]:
    filtered_df = df.dropna(subset=['salary_from_gross', 'salary_to_gross']).copy()
    filtered_df.loc[:, 'difference'] = filtered_df['salary_to_gross'] - filtered_df['salary_from_gross']
    condition = filtered_df['difference'] > 0.8 * filtered_df['salary_to_gross']
    filtered_df.loc[condition, ['salary_from_gross', 'difference']] = np.nan

    filtered_df = filtered_df.dropna(subset=['salary_from_gross', 'salary_to_gross'])

    filtered_df = filtered_df[filtered_df['salary_from_gross'] >= 1000]
    filtered_df = filtered_df[filtered_df['salary_to_gross'] >= 10000]

    filtered_df.loc[:, 'difference_ratio'] = np.where(filtered_df['salary_to_gross'] != 0, 
                                                      filtered_df['difference'] / filtered_df['salary_to_gross'], 
                                                      np.nan)

    perc25 = filtered_df['difference_ratio'].quantile(0.25)
    perc50 = filtered_df['difference_ratio'].quantile(0.50)
    perc75 = filtered_df['difference_ratio'].quantile(0.75)

    return perc25, perc50, perc75


def fill_na_salary(df: pd.DataFrame, coef) -> pd.DataFrame:
    result_df = df.copy()
    result_df['salary_to_gross'] = result_df['salary_to_gross'].fillna(result_df['salary_from_gross'] / (1-coef))
    result_df['salary_from_gross'] = result_df['salary_from_gross'].fillna(result_df['salary_to_gross'] * (1-coef))
    result_df['salary_average'] = result_df['salary_average'].fillna((result_df['salary_to_gross'] + result_df['salary_from_gross']) / 2)
    return result_df


def exctract_features(df: pd.DataFrame) -> pd.DataFrame:
    transformers = [
        ('schedule_name', OneHotEncoder(sparse_output=False, drop='first'), ['schedule__name']),
        ('grade', OneHotEncoder(sparse_output=False, drop='first'), ['grade'])
    ]

    pipeline = Pipeline(steps=[
        ('column_transformer', ColumnTransformer(transformers=transformers, remainder='passthrough'))
    ])

    display(pipeline)

    df_transformed = pipeline.fit_transform(df)
    column_names = pipeline.named_steps['column_transformer'].get_feature_names_out()
    transofrmed_cols = [x for x in column_names if 'remainder' not in x]
    column_names = [x.replace('remainder__', '') if x not in transofrmed_cols else x for x in column_names]
    
    df_new = pd.DataFrame(df_transformed, columns=column_names)
    df_new[transofrmed_cols] = df_new[transofrmed_cols].astype(float)
    
    return df_new

In [83]:
df_clear = get_clear_df_version(df_original)
print_df_info(df_clear)

Shape: (109404, 13)
--------------------------------------------------


,id,income_name,area__name,schedule__name,grade,key_skills__names,employer__name,languages__names,exchange_rate,salary_from_gross,salary_to_gross,salary_average,not_rur
137064,96177345,Менеджер по продажам,Брянск,Удаленная работа,Junior (1-3),"['B2B Продажи', 'Навыки продаж', 'Холодные про...",ProBusiness,[],1.0,113000.0,NaN,NaN,0
139261,96110261,Менеджер по продажам,Ростов-на-Дону,Полный день,Middle (3-6),"['Активные продажи', 'Навыки переговоров', 'По...",Спецхимагро,[],1.0,150000.0,NaN,NaN,0
111205,95776398,Менеджер по продажам,Воронеж,Удаленная работа,Intern (0-1),"['Ответственный', 'Целеустремленный', 'Желающи...",Кондратский Денис Романович,[],1.0,33900.0,90400.0,62150.0,0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 109404 entries, 2 to 175454
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 109404 non-null  int64  
 1   income_name        109404 non-null  object 
 2   area__name         109404 non-null  object 
 3   schedule__name     109404 non-null  object 
 4   grade              109404 non-null  object 
 5   key_skills__names  109404 non-null  object 
 6   employer__name     109404 non-null  object 
 7   languages__names   109404 non-null  object 
 8   exchange_rate      109404 non-null  float64
 9   salary_from_gross  102005 non-null  float64
 10  salary_to_gross    59847 non-null   float64
 11  salary_average     52448 non-null   float64
 12  not_rur            109404 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 11.7+ MB


None

--------------------------------------------------


id                       0
income_name              0
area__name               0
schedule__name           0
grade                    0
key_skills__names        0
employer__name           0
languages__names         0
exchange_rate            0
salary_from_gross     7399
salary_to_gross      49557
salary_average       56956
not_rur                  0
dtype: int64

--------------------------------------------------


In [84]:
perc25, perc50, perc75 = get_difference_percentiles(df_clear)
perc25, perc50, perc75

(0.20689655172413793, 0.375, 0.5294117647058825)

In [85]:
df_perc25 = fill_na_salary(df_clear, perc25)
print_df_info(df_perc25)

Shape: (109404, 13)
--------------------------------------------------


,id,income_name,area__name,schedule__name,grade,key_skills__names,employer__name,languages__names,exchange_rate,salary_from_gross,salary_to_gross,salary_average,not_rur
49201,94533310,Специалист технической поддержки,Екатеринбург,Полный день,Junior (1-3),"['Настройка ПО', 'Техническое обслуживание', '...",Парфюмерно-косметический супермаркет Золотое Я...,[],1.0,73450.0,92610.869565,83030.434783,0
11770,92167232,Аналитик,Кемерово,Сменный график,Junior (1-3),"['Пользователь ПК', 'Инвентаризация', 'Учет ос...","МАГНИТ, Розничная сеть",[],1.0,70000.0,70000.000000,70000.000000,0
114587,95424457,Менеджер по продажам,Нижний Новгород,Полный день,Junior (1-3),"['Грамотная речь', 'Консультирование клиентов'...",Дятьково,[],1.0,79100.0,99734.782609,89417.391304,0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 109404 entries, 2 to 175454
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 109404 non-null  int64  
 1   income_name        109404 non-null  object 
 2   area__name         109404 non-null  object 
 3   schedule__name     109404 non-null  object 
 4   grade              109404 non-null  object 
 5   key_skills__names  109404 non-null  object 
 6   employer__name     109404 non-null  object 
 7   languages__names   109404 non-null  object 
 8   exchange_rate      109404 non-null  float64
 9   salary_from_gross  109404 non-null  float64
 10  salary_to_gross    109404 non-null  float64
 11  salary_average     109404 non-null  float64
 12  not_rur            109404 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 11.7+ MB


None

--------------------------------------------------


id                   0
income_name          0
area__name           0
schedule__name       0
grade                0
key_skills__names    0
employer__name       0
languages__names     0
exchange_rate        0
salary_from_gross    0
salary_to_gross      0
salary_average       0
not_rur              0
dtype: int64

--------------------------------------------------


In [86]:
extracted_df_perc25 = exctract_features(df_perc25)
print_df_info(extracted_df_perc25)

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('schedule_name',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['schedule__name']),
                                                 ('grade',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['grade'])]))])

Shape: (109404, 18)
--------------------------------------------------


,schedule_name__schedule__name_Гибкий график,schedule_name__schedule__name_Полный день,schedule_name__schedule__name_Сменный график,schedule_name__schedule__name_Удаленная работа,grade__grade_Junior (1-3),grade__grade_Middle (3-6),grade__grade_Senior (>6),id,income_name,area__name,key_skills__names,employer__name,languages__names,exchange_rate,salary_from_gross,salary_to_gross,salary_average,not_rur
108836,0.0,1.0,0.0,0.0,0.0,1.0,0.0,97158844,Специалист технической поддержки,Москва,['Технические системы и средства безопасности'...,ПК ФЕНСИС,[],1.0,169500.0,169500.0,169500.0,0
79702,0.0,0.0,0.0,1.0,0.0,0.0,0.0,91620182,Менеджер по продажам,Иркутск,"['Активные продажи', 'Подготовка коммерческих ...",Мамаева Юлия Сергеевна,[],1.0,67800.0,101700.0,84750.0,0
27844,0.0,1.0,0.0,0.0,0.0,0.0,0.0,94546740,Менеджер по работе с клиентами,Йошкар-Ола,"['Продажи физическим лицам', 'Продажа кредитны...",Коммерческий банк «Хлынов»,[],1.0,49720.0,62690.434783,56205.217391,0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109404 entries, 0 to 109403
Data columns (total 18 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   schedule_name__schedule__name_Гибкий график     109404 non-null  float64
 1   schedule_name__schedule__name_Полный день       109404 non-null  float64
 2   schedule_name__schedule__name_Сменный график    109404 non-null  float64
 3   schedule_name__schedule__name_Удаленная работа  109404 non-null  float64
 4   grade__grade_Junior (1-3)                       109404 non-null  float64
 5   grade__grade_Middle (3-6)                       109404 non-null  float64
 6   grade__grade_Senior (>6)                        109404 non-null  float64
 7   id                                              109404 non-null  object 
 8   income_name                                     10940

None

--------------------------------------------------


schedule_name__schedule__name_Гибкий график       0
schedule_name__schedule__name_Полный день         0
schedule_name__schedule__name_Сменный график      0
schedule_name__schedule__name_Удаленная работа    0
grade__grade_Junior (1-3)                         0
grade__grade_Middle (3-6)                         0
grade__grade_Senior (>6)                          0
id                                                0
income_name                                       0
area__name                                        0
key_skills__names                                 0
employer__name                                    0
languages__names                                  0
exchange_rate                                     0
salary_from_gross                                 0
salary_to_gross                                   0
salary_average                                    0
not_rur                                           0
dtype: int64

--------------------------------------------------
